In [1]:
#Importing packages
#
# You can delete whatever package(s) you want..
#
import os
import json
import time
import requests
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



# Leave This code here and lookup at pandas documentation 
# if you need to know about chained assignments
pd.options.mode.chained_assignment = None  # default='warn'

# Because most of my data sets I have made can have up to 30+ columns and 20+ rows
# code below will increase pandas defaults for max rows 
# and columns that you can display in a Juptyer Notebook
pd.options.display.max_columns = 60
pd.options.display.max_rows = 90

In [2]:
# Get our block frost api key from the file
with open('keys.txt', 'r') as file:
    api_key = file.read().replace('\n', '')
api_key = api_key.split('=')[1]

In [3]:
from blockfrost import BlockFrostApi, ApiError, ApiUrls

api = BlockFrostApi(
	project_id=api_key,
	base_url=ApiUrls.mainnet.value,
)

In [4]:
try:
    health = api.health()
    print(health)   # prints object:    HealthResponse(is_healthy=True)
    health = api.health(return_type='json') # Can be useful if python wrapper is behind api version
    print(health)   # prints json:      {"is_healthy":True}
    health = api.health(return_type='pandas')
    print(health)   # prints Dataframe:         is_healthy
                    #                       0         True

    
    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        return_type='pandas'
    )
    print(account_rewards)  # prints 221
    print(len(account_rewards))  # prints 20

    account_rewards = api.account_rewards(
        stake_address='stake1ux3g2c9dx2nhhehyrezyxpkstartcqmu9hk63qgfkccw5rqttygt7',
        count=20,
        gather_pages=True, # will collect all pages
    )
    print(account_rewards[0].epoch)  # prints 221
    print(len(account_rewards))  # prints 57

    address = api.address(
        address='addr1qxqs59lphg8g6qndelq8xwqn60ag3aeyfcp33c2kdp46a09re5df3pzwwmyq946axfcejy5n4x0y99wqpgtp2gd0k09qsgy6pz')
    print(address.type)  # prints 'shelley'
    for amount in address.amount:
        print(amount.unit)  # prints 'lovelace'

except ApiError as e:
    print(e)

Namespace(is_healthy=True)
{'is_healthy': True}
   is_healthy
0        True
    epoch    amount                                            pool_id
0     221  16576013  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
1     222  17132278  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
2     223  15649019  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
3     224  17281719  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
4     225  18604433  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
5     226  12591619  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
6     227  17166845  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
7     228  17391311  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
8     231  13442322  pool15yyxtkhz64p7a8cnax9l7u82s9t9hdhyxsa3tdm97...
9     234   4412912  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
10    235   5175375  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
11    236   8040874  pool16kus5xvdysgmtjp0hhlwt72tsm0yn2zcn0a8wg9em...
1

# This code below is used to get all the Pool_ids (bech32 format) for all registered pools on Cardano

In [13]:
df = pd.DataFrame()

for i in range(1,100):
        try:
                pool_ids = api.pools(
                        gather_pages=True,
                        count=100,
                        page=i,)
                if len(pool_ids)==0:
                        break
                pools_list = pd.DataFrame(pool_ids)
                df = df.append(pools_list)
        except ApiError as e:
                print(e)


In [14]:
df

,0
0,pool1z5uqdk7dzdxaae5633fqfcu2eqzy3a3rgtuvy087f...
1,pool1pu5jlj4q9w9jlxeu370a3c9myx47md5j5m2str0na...
2,pool1c8k78ny3xvsfgenhf4yzvpzwgzxmz0t0um0h2xnn2...
3,pool1q80jjs53w0fx836n8g38gtdwr8ck5zre3da90peux...
4,pool1ddskftmsscw92d7vnj89pldwx5feegkgcmamgt5t0...
...,...
27,pool1mnuwy85gq5wvl426dwh4zz5laryatvrvh6sw8dlvq...
28,pool14glhqr7989jz4yu0hnytpw62xlq0666sy075ztufd...
29,pool1ttxtgplmy2v7t9s56gm3xxhsyxe68kpx0vu3utts0...
30,pool1jj674cqm2uaexhwjkuv23nav6egcwz9f5xls76nfh...


## Now let's get the list of retiring pools and then we will cross reference it with the list of all the pools and then remove any retiring pools

In [7]:
df2 = pd.DataFrame()

for i in range(1,50):
        try:
                retired_pools = api.pools_retiring(gather_pages=True, count=100, page=i)
                pools = pd.DataFrame(retired_pools)
                df2 = df2.append(pools)
        except ApiError as e:
                print(e)

In [8]:
df2

,0
0,Namespace(pool_id='pool1a3u8zuz7fqavxgl48s8k37...
1,Namespace(pool_id='pool14zx5y80c5m5x5hch6e57pc...
2,Namespace(pool_id='pool15eera5yr3eur7tz067wfpp...
3,Namespace(pool_id='pool1wqzdqsm02tust35dds3u7k...
4,Namespace(pool_id='pool1qgm942xc95w8f6qsx2vd35...
5,Namespace(pool_id='pool1ffuh4tz68rmv9tcdfqazlu...
6,Namespace(pool_id='pool1hkcy0k7fjlcwtuyh6wsdn2...
7,Namespace(pool_id='pool1pg9twqvv0e9ds3q4n9dcd9...


In [ ]:
df3 = pd.DataFrame()

for i in range(len(df)):
        